In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import os
from v4vapp_backend_v2.config.setup import InternalConfig
from v4vapp_backend_v2.database.db import get_mongodb_client_defaults
from v4vapp_backend_v2.actions.tracked_models import TrackedBaseModel

target_dir = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2/"
os.chdir(target_dir)
print("Current working directory:", os.getcwd())


In [ ]:
from v4vapp_backend_v2.accounting.balance_sheet import (
    generate_balance_sheet_pandas,
    get_account_balance_printout,
    get_ledger_dataframe,
    list_all_accounts,
)


CONFIG = InternalConfig(config_filename="devhive.config.yaml").config

db_client = get_mongodb_client_defaults()

TrackedBaseModel.db_client = db_client
ledger_df = await get_ledger_dataframe()
balance_sheet_dict = await generate_balance_sheet_pandas(df=ledger_df)


In [ ]:
all_accounts = await list_all_accounts()
print("All accounts:", all_accounts)
balances = {}
for account in all_accounts:
    print(f"Account: {account}")
    balances[str(account)] = await get_account_balance_printout(
        account=account, full_history=True, df=ledger_df
    )
for account, balance in balances.items():
    print(f"Balance for {account}:")
    print(balance)


In [ ]:
print(balance_sheet_dict)

In [ ]:
print(ledger_df)


In [ ]:
from v4vapp_backend_v2.accounting.balance_sheet import (
    balance_sheet_all_currencies_printout,
    balance_sheet_printout,
)


balance_sheet_currencies_str = balance_sheet_all_currencies_printout(balance_sheet_dict)
balance_sheet_str = balance_sheet_printout(balance_sheet_dict)
print(balance_sheet_currencies_str)
print(balance_sheet_str)

In [ ]:
from v4vapp_backend_v2.accounting.balance_sheet import get_ledger_entries

print("Ledger entries:")
ledger_entries = await get_ledger_entries()
for ledger_entry in ledger_entries:
    print(ledger_entry)

print("Balance sheet (currencies):")
print(balance_sheet_currencies_str)
print("Balance sheet:")
print(balance_sheet_str)

for account, balance in balances.items():
    print(f"Balance for {account}:")
    print(balance)

In [ ]:
print(balances["Customer Liability Hive (Liability) - Sub: v4vapp-test"])


In [ ]:
print(balances["Fee Income Lightning (Revenue) - Sub: umbrel"])

In [ ]:
print(balances["Customer Deposits Hive (Asset) - Sub: devser.v4vapp"])

In [ ]:
print(balances["Treasury Lightning (Asset) - Sub: umbrel"])


In [ ]:
print(balances["Treasury Lightning (Asset) - Sub: umbrel"])


In [ ]:
print(balance_sheet_dict)
